In [ ]:
# Convert 2015 SEMCOG Urbansim base year demographic data to ABM test inputs

## ABM HHs: HHID, TAZ, *TYPE, HINCP, *ADJINC, NP, *HHT, VEH (* new variables)
## ABM Persons: HHID, PERID, AGEP, SEX, *ESR, *WKHP, *WKW, *SCHG, *MIL, *PINCP(for GQ)
## note. both ABM HH and Person tables include GQ HH and Person records

## see document for the details of task purpose and methodology

# 04-22-2020
# fix member id: no duplicates in same HH
# document: provide detailed data dictionary
# household id: consider shorter int number(in the future)
# NAs: Set all NAs to -9
# HHT: for GQ records is set to 0
# HINCP: Use PINCP (personal income) for GQ records instead of  (household income). 
# HINCP adjustment: this dataset uses original HINCP and PINCP from PUMS, so not adjustment was implemented. The "income" variable in official 2015 model base year was adjusted. 
 


In [ ]:
import pandas as pd
import numpy as np
import math
import time
import os
from pandas_profiling import ProfileReport
from datetime import date
import random
random.seed(1)

In [ ]:
#input setup
# all data under ABM/test_data_012020/
infolder = 'inputs'
outfolder = 'outputs'
hdf_model = 'all_semcog_data_02-02-18.h5' # model base year, final version
hdf_syn = 'starter6_20171019-1526.h5' # synthesized results
tract_puma = 'https://www2.census.gov/geo/docs/maps-data/data/rel/2010_Census_Tract_to_2010_PUMA.txt'

pums_hhs = 'ss15hmi.csv'
pums_pps = 'ss15pmi.csv'
newhh_vars = ['TYPE', 'HINCP', 'ADJINC', 'HHT']
newperson_vars = ['ESR', 'WKHP', 'WKW', 'SCHG', 'MIL']
na_value = -9 #for fillna

#output files
outhhs = 'abm_hhs_{}.csv'.format(str(date.today()))
outpersons = 'abm_persons_{}.csv'.format(str(date.today()))
outgq = 'abm_gq_{}.csv'.format(str(date.today()))
outgq_hhs = 'abm_gq_hhs_{}.csv'.format(str(date.today()))

In [ ]:
stm = pd.HDFStore(os.path.join(infolder, hdf_model), 'r') # model inputs
sts = pd.HDFStore(os.path.join(infolder, hdf_syn), 'r') #original synthesized output, with matching household ID to model inputs but doesn't include adjusted/created HHs and Persons

#1. tract to puma cross-walk for MI
df_tract_puma = pd.read_csv(tract_puma)
df_tract_puma = df_tract_puma.loc[df_tract_puma.STATEFP == 26]

#2 associate building with other geographic units: county, tract, PUMA, etc
bldgeos = pd.merge(stm['buildings'][['parcel_id', 'b_zone_id']], stm['parcels'][['census_bg_id', 'county_id']], 
            left_on = 'parcel_id', right_index = True, how = 'left')
bldgeos['tract'] = bldgeos.census_bg_id//10
bldgeos = pd.merge(bldgeos.reset_index(), df_tract_puma[['COUNTYFP','TRACTCE','PUMA5CE']], 
           left_on = ['county_id', 'tract'], right_on = ['COUNTYFP','TRACTCE'], how='left').set_index('building_id')
bldgeos = bldgeos.rename(columns = {'b_zone_id': 'zone_id'})
bldgeos = bldgeos.fillna(na_value).astype(int)                                                     

In [ ]:
def find_matching_index(df1, df2, keycols):
    """ for every record in df1, identify and sample a matching record from df2
        df1: table to search (iterate rows)
        df2: table to sample from (query and sample)
        keycols: list of columns to be used as matching key, both df1 and df2 must have same keycols 
        df1 ands df2 must have valid index name. final table will have df2 index column joined to df1
    """
    t0 = time.time()

    v1, v2 = df1.index.name, df2.index.name
    assert((type(v1) == str) & (type(v2) == str) == True)  # make sure both index names exist
    print ('matching indices: df1:{} -- df1:{}'.format(v1, v2))
    df1 = df1.reset_index().set_index([v1] + keycols).sort_index() 
    qrylst = df1.index # assign searching sequence
    tcounts = len(qrylst)

    dfsample = []
    while (len(qrylst) > 0) and (len(keycols) > 0):
        df2 = df2.reset_index()
        df2 = df2.set_index(keycols).sort_index() #set index for query
        qrylst_remain = []  #if query cannot find a matching sample, store the line with 1 less key in this list
        for k in qrylst:
            try:
                dfsample.append([k[0], df2.loc[k[1:]].sample(1)[v2].values[0]])
                if len(dfsample) % 1000 == 0: #show progress
                    print ('\rworking [%1.1f%%]' % (len(dfsample)/tcounts * 100), end ="")
            except:
                qrylst_remain.append(k[:-1]) # store with 1 less key
        qrylst = qrylst_remain
        keycols = keycols[:-1] #drop one key
        print('\nkey = ', keycols, '| total successfully matched: ', len(dfsample), '| unmatched: ', len(qrylst))
        
    if len(qrylst_remain) > 0:  print("Done but still have unmatched records! ", qrylst_remain)
    df1 = pd.merge(df1.reset_index(), pd.DataFrame(dfsample, columns = [v1, v2]), 
                                        left_on = v1, right_on = v1, how = 'left' ).set_index(v1)
    print ("total time: %1.1f seconds" % (time.time()-t0))

    return df1

## Match PUMS samples for additional HHs in model inputs

In [ ]:
#process model HHs, attach bg_id, zone_id, county and PUMA
modhh = stm['households'].fillna(na_value)
modhh = pd.merge(modhh, bldgeos, left_on = 'building_id', right_index = True, how = 'left')
print('\nmodel index:  \t', modhh.index.name, '\ncolumns:   \t', modhh.columns.values)

#process synthesized HHS, attach county and PUMA
synhh = sts['sim_households'].fillna(na_value)
synhh.tract = synhh.tract.astype(int)
synhh['county_id'] = synhh.index.values//10000000
synhh = pd.merge(synhh.reset_index(), df_tract_puma[['COUNTYFP','TRACTCE','PUMA5CE']], 
                            left_on = ['county', 'tract'], right_on = ['COUNTYFP','TRACTCE'],                                                      how='left').set_index('household_id')
print('\nsynth index: \t', synhh.index.name, '\ncolumns: \t', synhh.columns.values)


In [ ]:
# enlist added HHs, that is all hhs in model inputs but not in original synthesis dataset( they were added in post-process)
modhh_match = modhh.loc[modhh.index.isin(synhh.index)]
modhh_added = modhh.loc[~modhh.index.isin(synhh.index)]
assert(len(modhh) == (len(modhh_match) + len(modhh_added)))
print(len(modhh_match), len(modhh_added))

In [ ]:
keycols = ['income', 'race_id', 'age_of_head','cars', 'county_id', 'PUMA5CE']

#process key col values for better matching process
for df in [synhh, modhh_added]: 
    df[keycols] = df[keycols].fillna(na_value)
    df[keycols] = df[keycols].round(0)
    df[keycols] = df[keycols].astype(int)
    print (df[keycols].head(2))


In [ ]:
#match synhh to additional model HHs
modhh_added = find_matching_index(modhh_added, synhh.reset_index().set_index('serialno'), keycols)

In [ ]:
#match = synhh.loc[synhh.index.isin(modhh.index)]
modhh.loc[modhh_match.index, 'serialno'] = synhh['serialno']
modhh.loc[modhh_added.index, 'serialno'] = modhh_added['serialno']
modhh.loc[modhh.serialno == -9] #verify if there's still added HHs with missing serialno

In [ ]:
# read HHT from 2015 PUMS dataset and attach them to model HHs
pumshhs = pd.read_csv(os.path.join(infolder, pums_hhs), usecols=['SERIALNO'] + newhh_vars)
modhh = pd.merge(modhh.reset_index(), pumshhs, left_on='serialno', right_on='SERIALNO', 
                                                                    how='left').set_index('household_id')
modhh.drop(['income', 'workers'], axis = 1, inplace = True) # drop income and workers for alternatives in ABM
print('save draft HHs to ' + os.path.join(outfolder, outhhs.replace('.csv', '_raw.csv')))
modhh.to_csv(os.path.join(outfolder, outhhs.replace('.csv', '_raw.csv')))

## Match PUMS samples for additional persons in model inputs

In [ ]:
# add extra variables from PUMS data to model persons
pumspps = pd.read_csv(os.path.join(infolder, pums_pps), usecols=['SERIALNO', 'SPORDER'] + newperson_vars)

modpp = stm['persons']
modpp_cols = modpp.columns
modpp = pd.merge(modpp.reset_index(), modhh[['serialno', 'county_id']], left_on='household_id',                                            right_index=True, how='left').set_index('person_id')  # join new serialno
modpp = pd.merge(modpp.reset_index(), pumspps, left_on=['serialno', 'member_id'], 
                    right_on=['SERIALNO', 'SPORDER'], how='left').set_index('person_id') #join new variables
modpp.fillna(na_value, inplace=True)

In [ ]:
modpp_added = modpp.loc[modpp.SERIALNO == -9] #persons without proper HH id after joining(new person only in model inputs) 
modpp_match = modpp.loc[modpp.SERIALNO != -9 ] #persons in original synthesis
assert(len(modpp) == (len(modpp_match) + len(modpp_added)))
print(len(modpp_match), len(modpp_added))

In [ ]:
keycols = ['county_id', 'worker', 'sex',  'race_id', 'age']
newid = modpp_added.index.name + '_add'
modpp_added.index.name = newid


In [ ]:
#additional person will find matches from existing person records
modpp_added = find_matching_index(modpp_added, modpp_match, keycols)

In [ ]:
#joining personal variables to additional persons using matched IDs
modpp_added = modpp_added.reset_index().set_index('person_id')
modpp_added[newperson_vars] = modpp_match[newperson_vars]
modpp_added = modpp_added.set_index(newid)
modpp.loc[modpp_added.index, newperson_vars] = modpp_added[newperson_vars]

In [ ]:
#fix duplicated member_id
modpp = modpp.reset_index()
modpp = modpp.sort_values(by=['household_id', 'member_id', 'person_id']) #sorting by existing ids
modpp['member_id'] = modpp.groupby('household_id').cumcount() + 1
modpp = modpp.set_index('person_id')

#fix duplicated partner records, update relate 1 to 10
modpp_relp = modpp.loc[(modpp.relate==1) & (modpp.duplicated(['household_id', 'relate']))].index 
modpp.loc[modpp_relp, 'relate'] = 10

In [ ]:
print('save draft persons to ' + os.path.join(outfolder, outpersons.replace('.csv', '_raw.csv')))
modpp.to_csv(os.path.join(outfolder, outpersons.replace('.csv', '_raw.csv')))


# Match PUMS to group quarter population

In [ ]:
#join county and PUMA to GQ pop
modgq = pd.merge(stm['/group_quarters'], bldgeos, left_on = 'building_id', right_index=True, how='left')
print(modgq.head(2))

In [ ]:
print('existing GQ types:', modgq.gq_code.unique())
# 601 MILITARY, NOT PRESENT IN THIS DATASET
# 401	oif	Other Institutional
# 501	csh	College/Student Housing
# 701	onf	Other NonInstitutional
# 789	HL	Homeless Population

modgq = modgq.loc[modgq.gq_code >= 501] #ABM only looks at noninstitutional GQ
modgq['TYPE'] = 3  #for GQ HH type
modgq.rename(columns={'PUMA5CE':'PUMA'}, inplace=True)
modgq['PUMA']= modgq['PUMA'].fillna(na_value).astype(int)
#modgq with TYPE, county_id, 

In [ ]:
#pumshhs = pd.read_csv('2015_synthpop_inputs/ss15hmi.csv', usecols=['SERIALNO', 'TYPE','NP', 'PUMA00','PUMA10'])

#2. PUMA 2000 to 2010 look up (one on one)
puma_00_10 = pd.read_excel('PUMA2000_PUMA2010_crosswalk.xls')[['State10','PUMA00', 'PUMA10', 'pPUMA00_Pop10']]
puma_00_10 = puma_00_10.loc[puma_00_10.State10 == 26].sort_values(['PUMA00', 'pPUMA00_Pop10'], ascending=False)
puma_00_10.drop_duplicates(subset ='PUMA00', keep='first', inplace= True)
puma_00_10= puma_00_10.set_index('PUMA00')

In [ ]:
#get PUMA id, filter by GQ pop and valid PUMAs
pumspps = pd.read_csv(os.path.join(infolder, pums_pps), usecols=['SERIALNO','SPORDER', 'RELP', 'AGEP', 'RAC1P', 'HISP','SEX', 'SCHG', 'WKHP', 'WKW', 'ESR','MIL', 'PINCP', 'PUMA00','PUMA10'])

pumspps = pumspps.set_index('PUMA00')
pumspps.update(puma_00_10[['PUMA10']]) #use lookup table to update PUMA10
pumspps =  pumspps.rename(columns={'PUMA10': 'PUMA'}).reset_index()
pumspps = pumspps.loc[pumspps.RELP.isin([16,17]) & pumspps.PUMA.isin(modgq.PUMA.unique())]
pumspps = pumspps.set_index('SERIALNO')

# prepare matching variables for PUMS samples
pumspps['TYPE'] = 3
pumspps.loc[pumspps.RELP ==16, 'TYPE'] = 2

pumspps['race_id'] = pumspps.RAC1P 
pumspps.loc[pumspps.RAC1P > 2, 'race_id'] = 4
pumspps.loc[pumspps.HISP > 1, 'race_id'] = 3

pumspps['age'] = pumspps.AGEP

In [ ]:
#match PUMS GQ records to model GQ

modgq = find_matching_index(modgq, pumspps, ['TYPE', 'race_id', 'age', 'PUMA'])

In [ ]:
modgq = pd.merge(modgq.reset_index(), pumspps, left_on = 'SERIALNO', right_index = True, how = 'left',                                                      suffixes = ('', '_y'))

modgq.rename(columns = {"RELP": "relate", "SEX":"sex", "SPORDER": "member_id"}, inplace = True)

modgq['person_id'] = range(2_000_000_000, 2_000_000_000 + len(modgq))
modgq['household_id'] = modgq['person_id']
modgq = modgq.set_index('person_id')

In [ ]:
# export GQ persons
print('save draft GQ persons to ' + os.path.join(outfolder, outgq.replace('.csv', '_raw.csv')))
modgq.to_csv(os.path.join(outfolder, outgq.replace('.csv', '_raw.csv')))

# Process group quarter households

In [ ]:
# covnert GQ persons to GQ HHs 
pumshhs = pd.read_csv(os.path.join(infolder, pums_hhs), usecols=['SERIALNO','VEH', 'NP','NOC'] + newhh_vars)
pumshhs = pumshhs.loc[pumshhs.TYPE > 1]
modgq_hhs = pd.merge(modgq, pumshhs, left_on = 'SERIALNO', right_on = 'SERIALNO', how = 'left', suffixes = ('', '_y') )


In [ ]:
# rename variables
modgq_hhs.rename(columns = {'age':'age_of_head', 
                            'VEH': 'cars', 
                            'NOC': 'children',
                            'NP': 'persons'}, inplace = True)
modgq_hhs = modgq_hhs.set_index('household_id')  
modgq_hhs['HHT'] = 0 #based on RSG suggestion
modgq_hhs['HINCP'] = modgq_hhs['PINCP'] 
modgq_hhs.drop('PINCP', axis=1, inplace=True)

In [ ]:
print('save draft GQ HHs to ' + os.path.join(outfolder, outgq_hhs.replace('.csv', '_raw.csv')))
modgq_hhs.to_csv(os.path.join(outfolder, outgq_hhs.replace('.csv', '_raw.csv')))
#list(set(stm['households'].columns) - set(['income', 'workers'])) + newhh_vars

# Combine household and GQ records


In [ ]:
# select needed variables for final outputs
hset = list(set(stm['households'].columns) - set(['income', 'workers'])) + newhh_vars
pset = list(set(stm['persons'].columns) - set(['worker'])) + newperson_vars

#final households and persons 
final_hhs = pd.concat([modhh[hset], modgq_hhs[hset]]).fillna(-9)
final_pps = pd.concat([modpp[pset], modgq[pset]]).fillna(-9)

print('save final HHs to ' + os.path.join(outfolder, outhhs))
final_hhs.to_csv(os.path.join(outfolder, outhhs))
print('save final Persons to ' + os.path.join(outfolder, outpersons))
final_pps.to_csv(os.path.join(outfolder, outpersons))

In [ ]:
#verification of data
# 1) any NA values
print(final_hhs.loc[final_hhs.isnull().any(axis=1)])
print(final_pps.loc[final_pps.isnull().any(axis=1)])

#2) member id
print(final_pps.loc[final_pps.duplicated(['household_id', 'member_id'])])

# 3) HHT values
print(final_hhs.HHT.unique())

#4) HINCP for GQ
print(final_hhs.loc[(final_hhs.HHT==0 ) & (final_hhs.HINCP<0 )].HINCP)

# (optional) compile ABM variables

In [ ]:
# update person and household worker with new definition ESR in [1,2,4,5] 
modpp.loc[modpp.ESR.isin([1,2,4,5]), 'worker'] = 1
modhh['workers'] = 0 #reset the values, cannot keep them
modhh['workers'] = modpp.groupby('household_id').worker.sum()

modpp['pemploy'] = 2 #part time
modpp.loc[modpp.age < 16, 'pemploy'] = 4  #under16
modpp.loc[(modpp.age >= 16) & (modpp.ESR.isin([3,6])) , 'pemploy'] = 3 #not employed
modpp.loc[(modpp.age >= 16) & (~modpp.ESR.isin([3,6])) & (modpp.WKHP >= 35) & (modpp.WKW.isin([1,2,3,4])), 'pemploy'] = 1  # full time

modpp.loc[(modpp.age >= 16) & (modpp.pemploy == 1), 'pstudent'] = 3 # not attending school
modpp.loc[(modpp.age < 16) & (modpp.pemploy == 1), 'pstudent'] = 1 # high school or lower
modpp.loc[modpp.SCHG.isnull() & (modpp.age >= 16), 'pstudent'] = 3
modpp.loc[modpp.SCHG.isnull() & (modpp.age < 16), 'pstudent'] = 1
modpp.loc[(modpp.pemploy != 1) & (modpp.SCHG >= 15) & (modpp.age >= 16), 'pstudent'] = 2# college or higher
modpp.loc[(modpp.pemploy != 1) & (modpp.SCHG >= 15) & (modpp.age < 16), 'pstudent'] = 1
modpp.loc[(modpp.pemploy != 1) & (modpp.SCHG.isin(range(1,15))) & (modpp.age <= 19), 'pstudent'] = 1
modpp.loc[(modpp.pemploy != 1) & (modpp.SCHG.isin(range(1,15))) & (modpp.age > 19), 'pstudent'] = 2

modpp.loc[modpp.pemploy == 1, 'ptype'] = 1
modpp.loc[(modpp.pemploy == 2) & (modpp.pstudent == 3), 'ptype'] = 2
modpp.loc[(modpp.age >= 65) & (modpp.pemploy.isin([3,4])) & (modpp.pstudent == 3), 
            'ptype'] = 5
modpp.loc[(modpp.age < 6) & (modpp.pemploy.isin([3,4])) & (modpp.pstudent == 3), 
            'ptype'] = 8
modpp.loc[((modpp.age >= 6) & (modpp.age <= 64)) & (modpp.pemploy.isin([3,4])) &                           (modpp.pstudent == 3) , 'ptype'] = 4
modpp.loc[(modpp.pemploy.isin([2, 3, 4])) & (modpp.pstudent == 2) , 'ptype'] = 3
modpp.loc[(modpp.age < 6) & (modpp.pemploy.isin([2,3,4])) & (modpp.pstudent == 1), 
            'ptype'] = 8
modpp.loc[(modpp.age >= 16) & (modpp.pemploy.isin([2,3,4])) & (modpp.pstudent == 1), 
            'ptype'] = 6
modpp.loc[((modpp.age >= 6) & (modpp.age < 16)) & (modpp.pemploy.isin([2,3,4])) &                       (modpp.pstudent == 1) , 'ptype'] = 7

In [ ]:
# RSG notes on ABM data inputs

# Household attributes:
# 1.	Hworkers: Yes, this is number of workers in the household based on each member’s Employment Status Recode (ESR). ESR is defined in PUMS as follows:
# Members with ESR as 1, 2, 4 and 5 are counted as workers
# 2.	HHT: Yes, this is the original PUMS field – Household/family type


# Person Type (ptype):
# 1.	The ptype code is defined using the following PUMS person-level variables:
# a.	ESR: Employment Status Recode (ESR)
# b.	WKHP: Usual hours worked per week past 12 months
# c.	WKW: Weeks worked during past 12 months
# d.	SCHG: Grade level attending
# e.	AGEP: Age

# 2.	The person type is derived from person’s age, employment status (pemploy) and student status (pstudent).
# 3.	The employment status is derived from ESR, WKHP, WKW and Age
# 4.	The student status is derived from SCHG, Age and employment status

# As long as we have ESR, WKHP, WKW, SCHG and AGEP in the person file, employment status, student status and person type can be derived.

# We have documented the person type coding process for ODOT. Please follow this link for a  detailed description of person type coding logic: https://github.com/RSGInc/SOABM/wiki/Person-Type-Coding-in-SOABM

# https://github.com/RSGInc/SOABM/wiki/Person-Type-Coding-in-SOABM
# PUMS variable definitions( year unkown * different from 2015 definitions see below)
# Employment status recode ESR
#     b .N/A (less than 16 years old)
#     1 .Civilian employed, at work
#     2 .Civilian employed, with a job but not at work
#     3 .Unemployed
#     4 .Armed forces, at work
#     5 .Armed forces, with a job but not at work
#     6 .Not in labor force

# WKHP
# Usual hours worked per week past 12 months
#     bb .N/A (less than 16 years old/did not work during the past 12 months)
#     01..98 .1 to 98 usual hours
#     99 .99 or more usual hours

# WKW
# Weeks worked during past 12 months
#     b .N/A (less than 16 years old/did not work during the past 12 months)
#     1 .50 to 52 weeks
#     2 .48 to 49 weeks
#     3 .40 to 47 weeks
#     4 .27 to 39 weeks
#     5 .14 to 26 weeks
#     6 .13 weeks or less

# ========================================================
# SCHG https://github.com/RSGInc/SOABM/wiki/Person-Type-Coding-in-SOABM
# Grade level attending
#     b .N/A (not attending school)
#     1 .Nursery school/preschool
#     2 .Kindergarten
#     3 .Grade 1 to grade 4
#     4 .Grade 5 to grade 8
#     5 .Grade 9 to grade 12
#     6 .College undergraduate
#     7 .Graduate or professional school
#     AGEP
#     Age
#     00 .Under 1 year
#     01..99 .1 to 99 years (Top-coded***)




# SCHG (2015 PUMS variable codes, SEMCOG model base year data)
# Grade level attending
#     bb .N/A (not attending school)
#     01 .Nursery school/preschool
#     02 .Kindergarten
#     03 .Grade 1
#     04 .Grade 2
#     05 .Grade 3
#     06 .Grade 4
#     07 .Grade 5
#     08 .Grade 6
#     09 .Grade 7
#     10 .Grade 8
#     11 .Grade 9
#     12 .Grade 10
#     13 .Grade 11
#     14 .Grade 12
#     15 .College undergraduate years (freshman to senior)
#     16 .Graduate or professional school beyond a bachelor's degree